<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Amazon_Price_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web スクレイピング （Amazon版）
【おことわり】プログラムでアクセスすると回線を切られるので、Selenium により Web ブラウザを遠隔操作することでアクセス、情報の入手を行います。



*   Selenium 入門 --> https://www.selenium.dev/ja/documentation/webdriver/getting_started/



# 1. Selenium による Web スクレイピング

### ライブラリのインストール

In [ ]:
import os
os.environ['https_proxy'] = 'http://cmproxy.nda.ac.jp:8080'
!pip install selenium --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

### ライブラリのインポート

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import re
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

#from bs4 import BeautifulSoup
import time

### Chrome Driver の設定

In [ ]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行したい場合は設定
#options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

### Web ページの取得

In [ ]:
url = 'https://www.amazon.co.jp'
query = 'パソコン'
#インスタンス化
driver = webdriver.Chrome(options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
# ページの取得
driver.get(url)

# 要素が表示されるまで待機
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="navBackToTop"]/div/span')))
time.sleep(3)

element = driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]')
element.send_keys("")
for i in query:
    element.send_keys(i)
    time.sleep(1)
element.send_keys(Keys.ENTER)

win_height = driver.execute_script("return window.innerHeight")

#スクロール開始位置の初期値（ページの先頭からスクロールを開始する）
last_top = 1

#スクロール前のページの高さを取得
last_height = driver.execute_script("return document.body.scrollHeight")

#スクロール開始位置を設定
top = last_top

#ページ最下部まで、徐々にスクロールしていく
while top < last_height:
    top += int(win_height * 0.1)
    driver.execute_script("window.scrollTo(0, %d)" % top)
    time.sleep(0.1)

prices = driver.find_elements(By.CLASS_NAME,"a-price-whole")
print(len(prices), '個の価格を取得')

### 価格情報の入手

In [ ]:
sum = 0
count = 0
for i in prices:
    print(i.text, ',', len(i.text))
    if len(i.text) > 0:
        sum += int(i.text.replace(',', ''))
        count += 1

print("平均価格：", sum / count, '円 (', sum, '円/', count, '個)')

### ブラウザを閉じる

In [ ]:
driver.quit()